In [114]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

In [115]:
names = ["Naive Bayes", "QDA","Decision Tree", "Random Forest", "Neural Net", "AdaBoost","Nearest Neighbors", "Linear SVM", "RBF SVM"]
classifiers =[GaussianNB(),QuadraticDiscriminantAnalysis(),DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=5),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),KNeighborsClassifier(5),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1)]
classifiers_A =[GaussianNB(),QuadraticDiscriminantAnalysis(),DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(activation='relu',alpha=.1),
    AdaBoostClassifier(),KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='kd_tree'),
    SVC(kernel="linear", C=1),
    SVC(gamma=2, C=1)]

In [116]:
path = "/home/jorgher/Documents/faceRec/CFEE/"
files = sorted(glob.glob(path+'*.csv'))
files = files[:7]

In [117]:
path = "/home/jorgher/Documents/faceRec/CFEE/"
files = sorted(glob.glob(path+'*.csv'))
files = files[:5]
#emotions = ["neutral",'happy','sad','fearful','angry','surprised','disgusted']
#emotions = [0,1,2,3,4,5,6]
emotions = [0,1,2,3,4]
dbs = []
for i,f in enumerate(files):
    db = pd.read_csv(f)
    db['file'] = db['file'].map(lambda x: str(x)[15:])
    db['file'] = db['file'].map(lambda x: str(x)[:-4]).astype(int)
    db['Subjet'] = db['file']
    db['Emotions'] = emotions[i]
    dbs.append(db)

db = (pd.concat(dbs[i][:] for i in range(len(files))))
db = db.sort_values('file')
#df['Emotion']=df['file']
db.drop(['file'], axis=1, inplace=True)

In [118]:
db.head()

,D1,D2,D3,D4,D5,D6,D7,A0,A1,A2,A3,A4,A5,Subjet,Emotions
45,22.360680,21.213203,25.298221,12.041595,14.035669,40.000000,91.049437,1.058044,0.294076,1.067170,0.278944,0.752528,1.219925,1,3
216,22.090722,20.024984,24.041631,9.000000,8.246211,18.110770,64.124878,0.971679,0.320525,1.165202,0.291506,0.526344,1.059376,1,4
92,33.241540,28.160256,32.310989,9.000000,9.055385,39.012818,94.191295,1.053966,0.262995,1.089529,0.408097,0.731317,1.121536,1,1
36,32.140317,28.017851,32.572995,12.000000,11.180340,35.014283,71.028163,1.178367,0.280184,1.011086,0.392952,0.869221,1.264622,1,0
68,21.095023,27.018512,30.364453,11.000000,10.049876,31.016125,70.178344,1.060248,0.344625,1.020170,0.261044,0.740912,1.250330,1,2


In [119]:
features = list(db.columns[:13])
Etype={'neutral': 0, 'happy': 1,
        'sad': 2,'fearful': 3,'angry': 4,
        'surprised': 5,'disgusted': 6}

In [120]:
db_data = db[features]

In [121]:
db_target = db['Emotions']

In [122]:
X_train, X_test, y_train, y_test = train_test_split(db_data, db_target, test_size=0.3, random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

In [123]:
X_train_transformed.shape, y_train.shape

((805, 13), (805,))

In [124]:
X_test_transformed.shape, y_test.shape

((345, 13), (345,))

In [125]:
for name, clf in zip(names, classifiers_A):
    print()
    print(name)
    clf.fit(X_train_transformed, y_train)
    cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
    scores = cross_val_score(clf, X_test_transformed, y_test, cv=cv)
    print(scores)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    Y_pred = clf.predict(X_test_transformed)
    exito = (list(y_test[:10]-Y_pred[:10]).count(0)/10)
    print("Porcentaje exito {} %".format(exito*100) )


Naive Bayes
[ 0.64423077  0.60576923  0.625       0.63461538  0.56730769  0.60576923
  0.67307692  0.66346154  0.65384615  0.65384615]
Accuracy: 0.63 (+/- 0.06)
Porcentaje exito 60.0 %

QDA
[ 0.66346154  0.54807692  0.61538462  0.57692308  0.57692308  0.65384615
  0.625       0.61538462  0.55769231  0.53846154]
Accuracy: 0.60 (+/- 0.08)
Porcentaje exito 70.0 %

Decision Tree
[ 0.51923077  0.5         0.49038462  0.55769231  0.52884615  0.53846154
  0.54807692  0.57692308  0.625       0.50961538]
Accuracy: 0.54 (+/- 0.08)
Porcentaje exito 60.0 %

Random Forest
[ 0.60576923  0.58653846  0.60576923  0.60576923  0.55769231  0.55769231
  0.58653846  0.63461538  0.54807692  0.61538462]
Accuracy: 0.59 (+/- 0.05)
Porcentaje exito 60.0 %

Neural Net


/home/jorgher/.virtualenvs/cv/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/jorgher/.virtualenvs/cv/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/jorgher/.virtualenvs/cv/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/jorgher/.virtualenvs/cv/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the op

[ 0.75        0.63461538  0.66346154  0.58653846  0.61538462  0.67307692
  0.70192308  0.72115385  0.66346154  0.67307692]
Accuracy: 0.67 (+/- 0.09)
Porcentaje exito 90.0 %

AdaBoost
[ 0.41346154  0.59615385  0.55769231  0.53846154  0.48076923  0.56730769
  0.55769231  0.55769231  0.5         0.39423077]
Accuracy: 0.52 (+/- 0.13)
Porcentaje exito 80.0 %

Nearest Neighbors
[ 0.60576923  0.53846154  0.53846154  0.64423077  0.55769231  0.59615385
  0.59615385  0.69230769  0.66346154  0.63461538]
Accuracy: 0.61 (+/- 0.10)
Porcentaje exito 50.0 %

Linear SVM
[ 0.67307692  0.66346154  0.65384615  0.625       0.61538462  0.65384615
  0.70192308  0.68269231  0.625       0.64423077]
Accuracy: 0.65 (+/- 0.05)
Porcentaje exito 60.0 %

RBF SVM
[ 0.21153846  0.16346154  0.20192308  0.26923077  0.25        0.20192308
  0.25        0.18269231  0.18269231  0.19230769]
Accuracy: 0.21 (+/- 0.07)
Porcentaje exito 40.0 %


In [101]:
# create some column headers
col_headers = ["col{}".format(i) for i in range(1, 4)]
aligned_row = ["{:^8}".format(i) for i in range(1, 4)]

# print the column headers first
print("{:^8} {:^8} {:^8}".format(*col_headers))
print("{:^8} {:^8} {:^8}".format(*aligned_row))

  col1     col2     col3  
   1        2        3    
